## Training In-hand Object Manipulation

In this notebook, we use AngoraPy to train an agent on in-hand object manipulation (IHOM). The notebook accompanies the example presented in the technical paper introducing AngoraPy but additionally annotates it with code.

We begin the notebook by importing ipyparallel and setting it up to allow usage of MPI with the notebook.

In [ ]:
from ipyparallel import Client, error, Cluster
client = Client(profile="mpi", block=True)

### The Task

In-hand object manipulation (IHOM) is a manual dexterity task. To simulate it, we use the hand model shipped with AngoraPy. It consists of $24$ joints and has its palm connected to a fixed socket via a joint with two degrees of freedom. Actuators are directly attached to the joints and apply control in terms of the absolute desired joint angles. Out of $24$ joints, $4$ are coupled. Thus, they cannot be controlled directly but move dependent on other joints. Accordingly, the motor plant has a total of $20$ degrees of freedom. In-hand object manipulation covers a broad category of tasks, but teaching it to an artificial agent requires a prototypical specification. Consistent with Andrychowicz et al (2018), we prototype the manipulation task as the in-hand reorientation of a cube whose faces are uniquely colored and labeled. A target reorientation is specified as an angle of rotation around a fixed point (the object's center) and achieved if the cube's rotation angle lies within $\eta$ units of the target angle; that is, their distance $d_g(t) \leq \eta$. To encourage stable behavior towards the end of a reorientation, we define a single episode as a chain of reorientations. Thus, the agent needs to learn manipulation in a manner that maintains sufficient control to enable it to perform the next reorientation from the endpoint of the previous. Per reorientation (i.e., goal), the agent is given $8$ seconds, and the total number of possible reorientations is capped at $50$. The $8s$-timer resets for every reached goal. The episode ends immediately when the cube is dropped, indicated by the cube center's $z$ position coming below that of the palm.

In AngoraPy, this task definition already exists in different variants. The respective environment ids begin with _HumanoidManipulateBlock_ and follow on _Discrete_ if they work on a discrete action space, and additionally _Asynchronous_ if they supply auxiliary information for an asymmetric value function. Since in our example, we want to do both, out environment id is _HumanoidManipulateBlockDiscreteAsynchronous-v0_. We build this environment and attach a state and reward normalization transformer to it, in the following.

In [ ]:
%%px
from angorapy import make_env
from angorapy.common.transformers import StateNormalizationTransformer, RewardNormalizationTransformer

environment = make_env("HumanoidManipulateBlockDiscreteAsynchronous-v0", transformers=[StateNormalizationTransformer, RewardNormalizationTransformer])

Next, we build our model. In AngoraPy, we do not work with an instance of a model itself, but with model building functions that create such models. This supports requirements of the backend. In real applications, users would likely want to define their own models. However, for testing purposes or proofs of concepts, standard model building functions can be created by the `get_model_builder()` function. It accepts an architecture name (one of __simple__, __deeper__, and __wider__) and a model type determining the memory cell of the model (one of __ffn__, __rnn__, __gru__, and __lstm__).

Additionally to the model architecture, we also need to choose a policy distribution. For IHOM, the `MultiCategoricalPolicyDistribution` has shown to be effective.

In [ ]:
%%px
from angorapy import get_model_builder
from angorapy.common.policies import MultiCategoricalPolicyDistribution

model_builder = get_model_builder(model="wider", model_type="lstm")
p_distribution = MultiCategoricalPolicyDistribution(env=environment)

This already wraps up the components necessary to build an agent. After instantiating it, we can immediately initiate training by calling the agents `drill()` method.

In [ ]:
%%px
from angorapy.agent import PPOAgent

agent = PPOAgent(model_builder, environment, distribution=p_distribution, workers=12, horizon=4096)

In [ ]:
%%px
agent.drill(
    100,
    epochs=3,
    batch_size=3072
)